In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd

In [2]:
df = pd.read_csv('./datasets/Bus_Stop_Shelter.csv')

In [67]:
df.shape

(3330, 18)

In [3]:
df.head()

,the_geom,BoroCode,BoroName,BoroCD,CounDist,AssemDist,StSenDist,CongDist,Shelter_ID,Corner,On_Street,Cross_Stre,Longitude,Latitude,NTAName,FEMAFldz,FEMAFldT,HrcEvac
0,POINT (-73.94874357365637 40.64085475597672),3,Brooklyn,317,45,42,21,9,BR02218,SW,AV D,NOSTRAND AV,-73.948745,40.640863,East Flatbush-Erasmus,X,AREA OF MINIMAL FLOOD HAZARD,5.0
1,POINT (-74.02283555116742 40.6292447605423),3,Brooklyn,310,43,46,26,11,BR02219,SW,5 AV,BAY RIDGE PKWY,-74.022837,40.629253,Bay Ridge,X,AREA OF MINIMAL FLOOD HAZARD,NaN
2,POINT (-73.92467258003798 40.67375874807616),3,Brooklyn,308,41,55,20,9,BR02220,SE,ST MARKS AV,BUFFALO AV,-73.924674,40.673767,Crown Heights (North),X,AREA OF MINIMAL FLOOD HAZARD,NaN
3,POINT (-73.9401345806071 40.578033768126936),3,Brooklyn,315,48,45,23,8,BR02221,SW,ORIENTAL BLVD,MACKENZIE ST,-73.940136,40.578042,Sheepshead Bay-Manhattan Beach-Gerritsen Beach,X,AREA OF MINIMAL FLOOD HAZARD,1.0
4,POINT (-73.93991658060635 40.57805676808165),3,Brooklyn,315,48,45,23,8,BR02222,SW,ORIENTAL BLVD,MACKENZIE ST,-73.939918,40.578065,Sheepshead Bay-Manhattan Beach-Gerritsen Beach,X,AREA OF MINIMAL FLOOD HAZARD,1.0


In [4]:
#drop all columns except for BoroCode, BoroName, and Longitude, Latitude
df = df[['NTAName', 'On_Street', 'BoroName', 'Longitude', 'Latitude']]

In [5]:
#get unique BoroName
df['BoroName'].unique()

array(['Brooklyn', 'Queens', 'Bronx', 'Manhattan', 'Staten Island'],
      dtype=object)

In [6]:
#drop all non Manhattan rows
df = df[df['BoroName'] == 'Manhattan']

In [7]:
df.shape

(720, 5)

In [8]:
df.head()

,NTAName,On_Street,BoroName,Longitude,Latitude
898,East Harlem (South),3 AV,Manhattan,-73.946160,40.789307
899,East Harlem (South),3 AV,Manhattan,-73.944332,40.791820
900,Harlem (South),ADAM CLAYTON POWELL BLVD,Manhattan,-73.952176,40.803386
901,Harlem (South),W 116 ST,Manhattan,-73.952078,40.802923
902,Harlem (South),FREDERICK DOUGLASS BLVD,Manhattan,-73.955667,40.804190


In [9]:
df['On_Street'].unique()

array(['3 AV', 'ADAM CLAYTON POWELL BLVD', 'W 116 ST',
       'FREDERICK DOUGLASS BLVD', 'W 225 ST', 'BROADWAY', 'WATER ST',
       'PEARL ST', 'FRANKFORT ST', 'MADISON ST', 'PARK ROW', 'W BROADWAY',
       'VARICK ST', '6 AV', 'W HOUSTON ST', 'BOWERY', 'GRAND ST',
       'Grand St', '1 AV', 'E 20 ST', 'E 23 ST', 'E 42 ST', 'E 49 ST',
       'E 72 ST', 'YORK AV', 'E 79 ST', 'MADISON AV', 'E 97 ST', '2 AV',
       'E 96 ST', 'ST NICHOLAS AV', '5 AV', 'E 14 ST', 'AV A',
       'AMSTERDAM AV', '8 AV', 'E HOUSTON ST', 'E 135 ST', 'COLUMBUS AV',
       'W 96 ST', 'W 135 ST', 'W 110 ST', 'EDWARD MORGAN PL', 'W 34 ST',
       'W 146 ST', 'WEST END AV', 'SOUTH END AV', 'W THAMES ST',
       'W 42 ST', 'AV C', 'AV D', 'W 50 ST', 'W 23 ST', 'E 116 ST',
       'E 10 ST', 'FT GEORGE AV', 'DELANCEY ST', 'FT WASHINGTON AV',
       '10 AV', 'W 57 ST', 'W 14 ST', '9 AV', 'HUDSON ST', 'W 66 ST',
       'LEXINGTON AV', 'E 57 ST', 'CHURCH ST', 'W 86 ST', 'W 65 ST',
       '7 AV', 'COLUMBIA ST', 'ESSEX ST

In [10]:
#assign taxi zones to locations.
# Load GeoJSON data into a GeoDataFrame
def load_geojson_gpd(filepath):
    return gpd.read_file(filepath)

# Function to find zones using spatial join in geopandas
def assign_zones(df, gdf):
    # Convert DataFrame to GeoDataFrame
    gdf_points = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.Longitude, df.Latitude))
    gdf_points.set_crs(gdf.crs, inplace=True)

    # Spatial join points to polygons
    joined = gpd.sjoin(gdf_points, gdf, how="left", predicate='within')
    return joined['location_id']



In [11]:
# Load GeoJSON data into a GeoDataFrame
geo_df = load_geojson_gpd('./datasets/NYC Taxi Zones.geojson')

# Assign zones
df['zone_id'] = assign_zones(df, geo_df)

df.head()

,NTAName,On_Street,BoroName,Longitude,Latitude,zone_id
898,East Harlem (South),3 AV,Manhattan,-73.946160,40.789307,75
899,East Harlem (South),3 AV,Manhattan,-73.944332,40.791820,75
900,Harlem (South),ADAM CLAYTON POWELL BLVD,Manhattan,-73.952176,40.803386,41
901,Harlem (South),W 116 ST,Manhattan,-73.952078,40.802923,41
902,Harlem (South),FREDERICK DOUGLASS BLVD,Manhattan,-73.955667,40.804190,41


In [12]:
#get unique zone ids
all_zones = df['zone_id'].unique()
#change type to str
all_zones = [str(x) for x in all_zones]

In [13]:
all_zones = set(all_zones)


In [14]:
zones_df = pd.read_csv('./datasets/zones_df.csv')

In [15]:
#get a list of the unique zone_ids as a list
our_zones = zones_df['zone_id'].unique()
our_zones = [str(x) for x in our_zones]
our_zones = set(our_zones)
our_zones

{'100',
 '107',
 '113',
 '114',
 '116',
 '12',
 '120',
 '125',
 '127',
 '128',
 '13',
 '137',
 '140',
 '141',
 '142',
 '143',
 '144',
 '148',
 '151',
 '152',
 '153',
 '158',
 '161',
 '162',
 '163',
 '164',
 '166',
 '170',
 '186',
 '194',
 '202',
 '209',
 '211',
 '224',
 '229',
 '230',
 '231',
 '232',
 '233',
 '234',
 '236',
 '237',
 '238',
 '239',
 '24',
 '243',
 '244',
 '246',
 '249',
 '261',
 '262',
 '263',
 '4',
 '41',
 '42',
 '43',
 '45',
 '48',
 '50',
 '68',
 '74',
 '75',
 '79',
 '87',
 '88',
 '90'}

In [16]:
zones_to_remove = all_zones - our_zones 
zones_to_remove

set()

In [17]:
len(our_zones)

66

In [18]:
#remove rows in df that have zone ids in the zones_to_remove set
df = df[~df['zone_id'].isin(zones_to_remove)]

In [19]:
#export to csv
df.to_csv('./datasets/bus_stop_shelters_with_zones.csv', index=False)

In [20]:
#aggregate bus stops by zone adding up the number of bus stops in each zone
df['count'] = 1
df = df.groupby('zone_id').count().reset_index()


In [21]:
#drop all columns except for zone_id and count
df = df[['zone_id', 'count']]

In [22]:
df.head()

,zone_id,count
0,107,15
1,113,2
2,114,3
3,116,18
4,120,4


In [23]:
#make sure we have all zones in the df
missing_zones_count = []
for zone in our_zones:
    if zone not in df['zone_id'].values:
        d = {'zone_id': zone, 'count': 0}
        missing_zones_count.append(d)

missing_zones_count = pd.DataFrame(missing_zones_count)
df = pd.concat([df, missing_zones_count])

In [24]:
#export to csv
df.to_csv('./datasets/count_bus_stop_shelters_by_zone.csv', index=False)